## A conversation with Andrew Ng
**Word Embedding** : for representing the semantics of a word
> Tensorflow에서 word embedding 기능 제공



## Introduction
**Embedding** : 특정 단어들과 그것과 연관된 단어들은 다차원 공간에서 벡터들로 군집화된다는 개념에서 나온 것
> ex. 특정 단어들은 '긍정' 방향을 향하고, 특정 단어들은 '부정' 방향을 향함

## The IMBD Dataset
텐서플로우의 데이터셋 라이브러리인 **TensorFlow Data Services (TFTS)**에는 다양한 데이터 셋들이 존재함 (image, audio, text 등)
> 그 중 text 데이터셋에 해당하는 **IMDB** ("**imdb_reviews**") : 긍정 or 부정으로 분류된 영화 리뷰 데이터들

## Looking into the details

In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
# !pip install -q tensorflow-datasets => 구글 코랩에는 이미 설치되어 있음!

#### [ cf. 2 methods of TensorFlow Datasets dataset loading ]
* 1. 
```py 
import tensorflow_datasets as tfds
tdds.load("imdb_reviews")
```
* 2. 
```py
import tensorflow as tf
tf.keras.datasets.imdb.load_data()
```

In [ ]:
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True) # imda data, metadata 반환 



import numpy as np

train_data, test_data = imdb['train'], imdb['test'] # imdb : train과 test set으로 나누어져 있음! (각 25000 씩)

training_sentences = []
training_labels = []

testing_sentences = []
training_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s, l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

# 1 : 긍정 label
# 0 : 부정 label

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
# tokenizing sentences

# set hyperparameters (to make easier to change)
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok) 
tokenizer.fit_on_texts(training_sentences)  
word_index = tokenizer.word_index  # training sentences로 만들어진 word_index
sequences = tokenizer.texts_to_sequences(training_sentences) 
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type) # padding and crop

testing_sentences = tokenizer.texts_to_sequences(testing_sentences) 
testing_padded = pad_sequences(testing_sequences, maxlen=max_length) 

In [ ]:
# define neural network ==> add Embedding!!

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), # 임베딩!! 텍스트 감정분석 핵심!!
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                            
])

## How can we use vectors?
* 비슷한 의미의 단어들은 비슷한 벡터들로 할당됨 (함께 모임)
* 모델 학습 : 벡터와 그것의 label을 학습하는 것
> ==> **embedding**
* embedding 결과 : 2차원 배열 (문장의 길이, embedding 차원 수)
  
    
* Flatten() 대신 **GlobalAveragePooling1D()** 사용 :  벡터 요소들 전체의 평균들을 flatten
> 좀 더 느리게 학습되지만, 더 정확한 결과 도출



In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), # 임베딩!
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                            
])

## More into the details

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


num_epochs = 10
model.fit(padded, # train_X
          training_labels_final, # train_Y
          epochs=num_epochs, # epochs
          validation_data=(testing_padded, testing_labels_final)) # val_set or callbacks

embedding 층의 학습 결과 확인 (weights)

In [ ]:
e = model.layeres[0]
weights = e.get_weights()[0] # embedding 층 : 0번째
print(weights.shape) # shape : (vocab_size, embedding_dim) ==> (10000, 16)

embedding 층의 weights.shape : (10000, 16)
> 즉, 말뭉치에 총 10000개의 단어들 존재, 16차원 배열로 embedding 작업 중!

In [ ]:
# reverse word index (키-값 순서 바꾸기) ==> to be able to plot it
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) 

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
# 위에서 reverse한 dict 이용! (숫자 인덱스로 접근)
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n") # 메타 데이터 파일
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n") # 벡터 파일
out_v.close()
out_m.close()